In [ ]:
import arcpy
import requests
import os
import zipfile
import io

In [ ]:
working_dir = r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code'
working_dir

In [ ]:
mn_geo = r'https://resources.gisdata.mn.gov/pub/'
mn_geo

## Bring in data

In [ ]:
def retrieve_data(mn_geo):
    output = requests.post(mn_geo)
    slashstuff = output.content
    zipp = zipfile.ZipFile(io.BytesIO(slashstuff))
    zipp.extractall(r'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code')

In [ ]:
# correct way
Mn_dem = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
Mn_counties = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip'
NLCD_roads_bridges = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
MN_NLCD = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
Streams = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_strahler_stream_order/shp_water_strahler_stream_order.zip'

retrieve_data(Mn_dem)
retrieve_data(Mn_counties)
retrieve_data(NLCD_roads_bridges)
retrieve_data(MN_NLCD)
retrieve_data(Streams)

### get study extent from mn counties 

arcpy.conversion.FeatureClassToFeatureClass(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb", "studyextent_counties", "CTY_NAME = 'Winona' Or CTY_NAME = 'Wabasha' Or CTY_NAME = 'Olmsted'", r'AREA "AREA" true true false 19 Double 0 0,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,AREA,-1,-1;PERIMETER "PERIMETER" true true false 19 Double 0 0,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,PERIMETER,-1,-1;CTYONLY_ "CTYONLY_" true true false 19 Double 0 0,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,CTYONLY_,-1,-1;CTYONLY_ID "CTYONLY_ID" true true false 19 Double 0 0,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,CTYONLY_ID,-1,-1;COUN "COUN" true true false 4 Short 0 4,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,COUN,-1,-1;CTY_NAME "CTY_NAME" true true false 20 Text 0 0,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,CTY_NAME,0,20;CTY_ABBR "CTY_ABBR" true true false 4 Text 0 0,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,CTY_ABBR,0,4;CTY_FIPS "CTY_FIPS" true true false 4 Short 0 4,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,CTY_FIPS,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\mn_county_boundaries.shp,Shape_Area,-1,-1', '')

In [ ]:
# merging the 3 counties into 1 polygon so I have my study extent
arcpy.management.Dissolve(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\studyextent_counties", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\mn_studyextent", None, None, "SINGLE_PART", "DISSOLVE_LINES", '')

### dealing with road data

In [ ]:
# get the raster in the study extent
se_roads = arcpy.sa.ExtractByMask(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\NLCD_2019_Land_Cover_Impervious_Descriptor.tif", "mn_studyextent", "INSIDE", '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); se_roads.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\se_roads")

### Dealing with DEM/slope

In [ ]:
# get the raster in the study extent
se_dem = arcpy.sa.ExtractByMask(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m", "mn_studyextent", "INSIDE", '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); se_dem.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\se_dem")

In [ ]:
# get slope in percent rise
se_slope = arcpy.sa.Slope("se_dem", "PERCENT_RISE", 1, "PLANAR", "METER"); se_slope.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\se_slope")

In [ ]:
# reclass slope where roads are 1 and others are 10
reclass_slope = arcpy.sa.Reclassify("se_slope", "VALUE", "0 3 1;3 6 2;6 12 3;12 15 4;15 18 8;18 21 9;21 24 10;24 27 10;27 30 10;30 533.479797 10", "DATA"); reclass_slope.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\reclass_slope")

## Deal with landcover

In [ ]:
# get the raster in the study extent
se_lc = arcpy.sa.ExtractByMask(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\NLCD_2019_Land_Cover.tif", "mn_studyextent", "INSIDE", '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); se_lc.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\se_lc")

In [ ]:
# reclass landcover
se_lc = arcpy.sa.Reclassify("se_lc", "NLCD_Land", "'Open Water' 10;'Developed, Open Space' 2;'Developed, Low Intensity' 2;'Developed, Medium Intensity' 2;'Developed, High Intensity' 2;'Barren Land' 5;'Deciduous Forest' 7;'Evergreen Forest' 7;'Mixed Forest' 7;Shrub/Scrub 7;Herbaceous 7;Hay/Pasture 9;'Cultivated Crops' 9;'Woody Wetlands' 9;'Emergent Herbaceous Wetlands' 10", "DATA"); se_lc.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\reclass_lc")

## Deal with streams

In [ ]:
# get the vector in the study extent
arcpy.analysis.Clip(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\streams_with_strahler_stream_order.shp", "mn_studyextent", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\se_streams", None)

In [ ]:
# reclassify 
reclass_stream = arcpy.sa.Reclassify("stream_raster", "Value", "1 1;2 2;3 8;4 8;5 10;6 10;8 10", "DATA"); reclass_stream.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\reclass_stream")

In [ ]:
# get nodata to be zero data # this is working when I run it through arcpro, but not through arcpy

master_stream = arcpy.sa.RasterCalculator('Con(IsNull("reclass_stream"),1, "reclass_stream")'); master_stream.save(r"c:\Users\swimb\documents\UMN\fall2022\GIS5571\Lab2\arc1lab2_code\arc1lab2_code.gdb\master_stream")

## weighted overlay/cost surface

In [ ]:
cost_surface = arcpy.sa.WeightedOverlay(r"('C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\master_stream' 25 'Value' (0 1; 1 1; 2 2; 8 9; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\reclass_lc' 25 'Value' (2 2; 5 6; 7 8; 9 10; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\reclass_slope' 25 'Value' (1 1; 2 2; 3 3; 4 4; 8 9; 9 10; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\reclass_roads' 25 'Value' (1 1; 2 2; 3 3; 7 8; 10 10; NODATA NODATA));1 10 1"); cost_surface.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\cost_surface")

## bring in dory start and picnic point

In [ ]:
# dory start
arcpy.management.XYTableToPoint(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\Start.csv", r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\dory_start", "Long", "Lat", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

In [ ]:
## cost distance and cost direction. This was only giving me cost distance, not cost direction through arcpy. had to run through arcpro to get cost direction
cost_surface_outs = arcpy.sa.CostDistance("dory_start", "cost_surface", None, r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\direction_surface", None, None, None, None, ''); cost_surface_outs.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\distance_surface")

## final step: cost path tool

In [ ]:
# optimal route
optimal_route = arcpy.sa.CostPath("dory_end", "distance_surface", "direction_surface", "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); optimal_route.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\arc1Lab2_Code\arc1Lab2_Code.gdb\optimalpath")

In [1]:
# weighted overlay cost surface with farm fields
out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\reclass_stre' 20 'VALUE' (0 1; 2 2; 3 3; 7 10; 9 10; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\Reclass_se_n1' 40 'Value' (1 1; 4 2; 5 2; 8 2; 9 2; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\reclass_road' 20 'VALUE' (1 1; 2 2; 3 3; 7 8; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\Reclass_slop2' 20 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 6; 8 9; 10 10; NODATA NODATA));1 10 1"); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\Weighted_lc")

In [ ]:
out_distance_raster = arcpy.sa.CostDistance("start", "Weighted_LC", None, r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\CostDirectionLC", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\costdistanceLC")

In [ ]:
# optimal route for landcover
out_raster = arcpy.sa.CostPath("end", "costdistanceLC", "CostDirectionLC", "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\optimalpathLC")

In [ ]:
# weighted overlay with slope. optimal route for slope
out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\reclass_stre' 20 'VALUE' (0 1; 2 2; 3 3; 7 7; 9 9; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\Reclass_se_n1' 20 'Value' (1 1; 4 3; 5 5; 8 8; 9 9; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\noncodeway\data\reclass_road' 10 'VALUE' (1 1; 2 2; 3 3; 7 7; 10 10; NODATA NODATA); 'C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\Reclass_slop2' 50 'Value' (0 8; 1 7; 2 6; 3 5; 4 4; 5 3; 8 2; 10 1; NODATA NODATA));1 10 1"); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\Weighted_slope")
out_distance_raster = arcpy.sa.CostDistance("start", "Weighted_slope", None, r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\CostDirectionSLOPE", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\costdistanceSLOPE")
out_raster = arcpy.sa.CostPath("end", "costdistanceSLOPE", "CostDirectionSLOPE", "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save(r"C:\Users\swimb\Documents\UMN\fall2022\GIS5571\Lab2\Arc1_lab2_arcpro\Arc1_lab2_arcpro.gdb\optimalpathSLOPE")